In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
import sys
sys.path.append('../../../code/utils/')
sys.path.append('../../../code/feature/')
import data_utils as du
import perf_utils as pu
import gc
from scipy.sparse import find

In [2]:
# Here we should probably define the auto-encoder
class AutoEncoder(nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 32),
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.Tanh(),
            nn.Linear(128, input_size),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [3]:
torch.manual_seed(20180429)    # reproducible
# Hyper Parameters
EPOCH = 100                                       #Haven't figured out, let these just be here
BATCH_SIZE = 256
LR = 5e-5         # learning rate

In [5]:
df_train = du.load_raw_data("train")
gc.collect()
print('Start formatting data: {}'.format(pu.get_time_str()))
X, y = du.get_set(df_train, test = False, features_u_want = ['house', 'interest2', 'kw1', 'kw2',  'appIdInstall'], a_features_u_want = ['aid', 'productId'])
print('End formatting data: {}'.format(pu.get_time_str()))
print(X[0])
del df_train
gc.collect()

Start formatting data: 09:44:13
End formatting data: 09:46:55
  (0, 1)	1
  (0, 82)	1
  (0, 110016)	1
  (0, 164008)	1
  (0, 209053)	1
  (0, 214665)	1
  (0, 238362)	1
  (0, 309188)	1
  (0, 374044)	1
  (0, 374098)	1
  (0, 374218)	1


56

In [8]:
with pu.profiler("Setting up autoencoder"):
    autoencoder = AutoEncoder(X.shape[1])
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR, weight_decay = 1e-5)
    criterion = nn.MSELoss()

[09:47:53] Finish Setting up autoencoder. △M: -364.05MB. △T: 1.1 seconds.


In [ ]:
y = torch.Tensor(y)
for epoch in range(EPOCH):
    batch = []
    for i in range(X.shape[0]):
        if(i%BATCH_SIZE!=0 or i==0):
            batch.append(X[i].toarray())
        else:
            encoded, decoded = autoencoder(torch.FloatTensor(batch))
            loss = criterion(decoded.view(1,-1), torch.Tensor(batch).view(1,-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batch = []
            # ===================log========================
            print('epoch [{}/{}], samples[{}/{}], loss:{:.8f}'
                  .format(epoch + 1, EPOCH, i ,X.shape[0], loss.item()))

epoch [1/100], samples[256/8798814], loss:0.00506179
epoch [1/100], samples[512/8798814], loss:0.00504267
epoch [1/100], samples[768/8798814], loss:0.00499937
epoch [1/100], samples[1024/8798814], loss:0.00496282
epoch [1/100], samples[1280/8798814], loss:0.00492862
